<a href="https://colab.research.google.com/github/gowripriya15/Stock_price_prediction/blob/main/LSTM_FB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**STOCK_PRICE_PREDICTION_FOR_LARGE_CAP_TECH_COMPANIES**
In this project,we will predict the stock price for various companies with the help of historical data and news-headlines by applying the LSTM model(Long Short-Term Memory).

**IMPORTING REQUIRED LIBRARIES**

In [ ]:
import math
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
import matplotlib.pyplot as plt
plt.style.use('seaborn')

**UPLOADING DATASETS**

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
import io
fb_data=pd.read_csv(io.BytesIO(uploaded['facebook_merge_news_headlines.csv']))
fb_data = pd.read_csv("facebook_merge_news_headlines.csv", parse_dates=['Date'])
fb_data





In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:

df=pd.read_csv(io.BytesIO(uploaded['HistoricalData_FB.csv']))

**CHECKING NULL VALUES**

In [ ]:
import seaborn as sns
sns.heatmap(fb_data.isnull(),cmap='plasma')

In [ ]:
fb_data.sort_index(inplace=True)

In [ ]:
df = pd.read_csv("HistoricalData_FB.csv", parse_dates=['Date'])
df.head()

In [ ]:
df.rename(columns={"Close/Last":"Close"},inplace = True)



In [ ]:
df.sort_values('Date', inplace=True)

In [ ]:
df.reset_index()

In [ ]:
df.columns

**MERGING THE HISTORICAL DATASET AND NEWS_HEADLINES DATASET**

In [ ]:
data = pd.merge(fb_data, df, how='inner', on=['Date'])

In [ ]:
data

In [ ]:
data["Close"] = data["Close"].str.replace('$', '').astype(float)
data["Open"] = data["Open"].str.replace('$', '').astype(float)
data["High"] = data["High"].str.replace('$', '').astype(float)
data["Low"] = data["Low"].str.replace('$', '').astype(float)

In [ ]:
data.head()

In [ ]:
data.reset_index(inplace=True)
data_1=data[["Date","Close"]]

In [ ]:
data_1=data[["Date","Close"]]

In [ ]:
data_1=data_1.rename(columns={"Date": "ds", "Close": "y"})

In [ ]:
data_1.head()

**Prophet model**:Prophet is a procedure for forecasting time series data based on an additive model where non-linear trends are fit with yearly, weekly, and daily seasonality, plus holiday effects. It works best with time series that have strong seasonal effects and several seasons of historical data.

In [ ]:
from fbprophet import Prophet
m = Prophet(daily_seasonality = True) # the Prophet class (model)
m.fit(data_1) 

In [ ]:
future = m.make_future_dataframe(periods=365) #we need to specify the number of days in future
prediction = m.predict(future)
m.plot(prediction)
plt.title("Prediction of the Facebook Stock Price using the Prophet")
plt.xlabel("Date")
plt.ylabel("Close Stock Price")
plt.show()

In [ ]:
m.plot_components(prediction)
plt.show()

In [ ]:
data.set_index("Date", inplace = True)
data.head()

In [ ]:
data.sort_index(ascending=True,inplace=True)
data.head()

In [ ]:
data.shape

In [ ]:
plt.figure(figsize=(16,9))
plt.title('Close Price History of Facebook', fontsize = 20)
plt.plot(data['Close'])
plt.xlabel('Date', fontsize = 16, color = 'g')
plt.ylabel('Close Price USD ($)', fontsize = 16, color = 'g')
plt.xticks(fontsize = 12)
plt.yticks(fontsize = 12)
plt.show()

In [ ]:
import os

In [ ]:
data.to_csv('NewsandHistoricaldata.csv')

In [ ]:
os.getcwd()

**SENTIMENT ANALYSIS**

In [ ]:
data['compound'] = ''
data['negative'] = ''
data['neutral'] = ''
data['positive'] = ''
data.head()

In [ ]:
import nltk
nltk.download('vader_lexicon')

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import unicodedata


sid = SentimentIntensityAnalyzer()

data['compound'] = data['Headlines'].apply(lambda x: sid.polarity_scores(x)['compound'])
data['negative'] = data['Headlines'].apply(lambda x: sid.polarity_scores(x)['neg'])
data['neutral'] = data['Headlines'].apply(lambda x: sid.polarity_scores(x)['neu'])
data['positive'] = data['Headlines'].apply(lambda x: sid.polarity_scores(x)['pos'])


data.head()

In [ ]:
data['label'] = 0
data.loc[data['compound'] > 0.2, 'label'] = 1
data.loc[data['compound'] < -0.2, 'label'] = -1
data

In [ ]:
data.drop(['label'], axis = 1)

In [ ]:
data.columns

In [ ]:
data = data[['Close', 'compound', 'negative', 'neutral', 'positive', 'Open', 'High', 'Low', 'Volume']]

data.head()

In [ ]:
stockdata=data.to_csv('stockdata.csv')

In [ ]:
stockdata = pd.read_csv('stockdata.csv', index_col = False)

stockdata.rename(columns={'Unnamed: 0':'Date'}, inplace = True)

stockdata.set_index('Date', inplace=True)

stockdata.head()

In [ ]:
stockdata.shape

In [ ]:
stockdata.isna().any()

In [ ]:
stockdata.describe(include='all')

In [ ]:
stockdata.info()

In [ ]:
close_price = stockdata.iloc[:,0] 

print("Close Price:")
close_price

In [ ]:
need_data = pd.concat([close_price], axis=1)


need_data.columns = ['close_price']  
need_data.tail(20)

In [ ]:
dataset = need_data.values

In [ ]:
training_data_len = math.ceil(len(dataset)*.8)
training_data_len

In [ ]:
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(dataset)

print(scaled_data)

In [ ]:
train_data = scaled_data[0:training_data_len , :]

x_train = []
y_train = []
past_days = 20
for i in range(past_days, len(train_data)):
    x_train.append(train_data[i-past_days:i, :])
    y_train.append(train_data[i, 0])
    if i<= past_days:
        print(x_train)
        print(y_train)
        print()

In [ ]:
x_train, y_train = np.array(x_train), np.array(y_train)

In [ ]:
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
x_train

In [ ]:
print(x_train.shape)
print(y_train.shape)

**APPLYING THE LSTM MODEL**

In [ ]:
import keras.models
import tensorflow as tf
tf.random.set_seed(0)
model = Sequential()
model.add(LSTM(160, return_sequences=True, activation = "tanh", input_shape= (x_train.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(80, return_sequences= False, activation = "tanh"))
model.add(Dense(60))
model.add(Dense(1))

model.summary()

In [ ]:
model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
from keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor = 'val_loss', mode = 'min',verbose = 1,patience = 5)

history = model.fit(x_train, y_train, batch_size = 64, epochs = 100, verbose = 1, validation_split = 0.2, callbacks = [early_stop])

In [ ]:
loss = pd.DataFrame(model.history.history)
loss.plot()

In [ ]:
test_data = scaled_data[training_data_len - past_days: , :]

x_test = []
y_test = dataset[training_data_len:, 0]
for i in range(past_days, len(test_data)):
    x_test.append(test_data[i-past_days:i, :])

In [ ]:
x_test = np.array(x_test)

In [ ]:
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

In [ ]:
predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions)

In [ ]:
predictions

In [ ]:
rmse = np.sqrt(np.mean(predictions - y_test)**2)
print("Root Mean Squared Error = ", rmse)

In [ ]:
import copy
valid = copy.deepcopy(data[training_data_len:])
valid['Predictions'] = predictions
plt.figure(figsize=(16,9))
plt.title('Model')
plt.xlabel('Date', fontsize= 15, color = 'g')
plt.ylabel('Close Price USD ($)', fontsize = 15, color = 'g')
plt.plot(valid['Close'], color = 'blue', label = 'Actual Close Price')
plt.plot(valid['Predictions'],  color = 'red', label = 'Predicted Close Price')
plt.legend(loc = 'lower right', fontsize = 12)
plt.xticks(fontsize = 12)
plt.yticks(fontsize = 12)
plt.show()

In [ ]:
valid

In [ ]:
import datetime

last_day = data.index[-1]
next_day = last_day + datetime.timedelta(days = 1)
print(next_day)

In [ ]:
new_df = data.filter(['Close'])
last_20_days = new_df[-20: ].values
last_20_days_scaled = scaler.transform(last_20_days)

X_test = []
X_test.append(last_20_days_scaled)

X_test = np.array(X_test)

X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

pred_price = model.predict(X_test)

pred_price = scaler.inverse_transform(pred_price)
print('Predicted Close Price for {} '.format(next_day) + ' = ', pred_price)

In [ ]:
previous = pred_price
previous_pred_price = new_df.at[str(last_day), 'Close']

In [ ]:
diff=(float)(pred_price - previous_pred_price)
if(diff < 0):
  print("percentage decrease = ",round(((- (diff)/previous_pred_price)*100),4))
else:
  print("percentage increase = ",round((( (diff)/previous_pred_price)*100),4))

In [ ]:
train = data[:training_data_len]
valid = data[training_data_len:]
valid['Predictions'] = predictions
plt.figure(figsize=(13,7))
plt.title('Model')
plt.xlabel('Date', fontsize=18)
plt.ylabel('Close Price ', fontsize=18)
plt.plot(train['Close'])
plt.plot(valid[['Close', 'Predictions']])
plt.legend(['Train', 'Val', 'Predictions'], loc='lower right')
plt.show()

In [ ]:
model.save('model.h5')